In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType, sum, max, col, concat, lit, monotonically_increasing_id
import sys
import os



from pyspark import Row


from datetime import datetime,timedelta

from fbprophet import Prophet
import pandas as pd
import numpy as np

In [6]:
def dfZipWithIndex (df, offset=1, colName="rowId"):
    '''
        Enumerates dataframe rows is native order, like rdd.ZipWithIndex(), but on a dataframe 
        and preserves a schema

        :param df: source dataframe
        :param offset: adjustment to zipWithIndex()'s index
        :param colName: name of the index column
    '''

    new_schema = StructType(
                    [StructField(colName,LongType(),True)]        # new added field in front
                    + df.schema.fields                            # previous schema
                )

    zipped_rdd = df.rdd.zipWithIndex()

    new_rdd = zipped_rdd.map(lambda args: ([args[1] + offset] + list(args[0])))

    return spark.createDataFrame(new_rdd, new_schema)

In [7]:
schema = StructType([
        StructField("ds", DateType(), True),
        StructField("yhat", DoubleType(), True)
    ])

In [8]:
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def fit_pandas_udf(df):
    """
    :param df: Dataframe (train + test data)
    :return: predictions as defined in the output schema
    """

    def train_fitted_prophet(df, cutoff):
        
        names = df.columns
        
        #train
        ts_train = (df
                    .query('id <= @cutoff')
                    .rename(columns={names[1]: 'ds', names[2]: 'y'})
                    .sort_values('ds')
                    )[['ds','y']]
        
        print(ts_train.columns)
        
        
        # test
        ts_test = (df
                   .query('id > @cutoff')
                   .rename(columns={names[1]: 'ds', names[2]: 'y'})
                   .sort_values('ds')
                   .assign(ds=lambda x: pd.to_datetime(x["ds"]))
                   .drop('y', axis=1)
                   )[['ds']]
        
        print(ts_test.columns)

 

        # init model
        m = Prophet(yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True)
        m.fit(ts_train)
        
        

        # to date
        
        # at this step we predict the future and we get plenty of additional columns be cautious
        ts_hat = (m.predict(ts_test)[["ds", "yhat"]]
                  .assign(ds=lambda x: pd.to_datetime(x["ds"]))
                  ).merge(ts_test, on=["ds"], how="left")  
        

        return pd.DataFrame(ts_hat, columns=schema.fieldNames())

    return train_fitted_prophet(df, cutoff)

In [9]:
if __name__ == '__main__':
    spark = (SparkSession
             .builder
             .appName("forecasting")
             .getOrCreate()
             #.config('spark.sql.execution.arrow.enable', 'true')
             )
    
    data = (spark
                .read
                .format("csv")
                .option('header', 'true')
                .option('inferSchema','true')
                .load('Downloads/AEP_hourly.csv')
            )
    
    data.createOrReplaceTempView("data")
    data = spark.sql(f"SELECT LEFT(Datetime,10) AS Datetime, {data.columns[1]}  FROM data")
    data = data.groupBy("Datetime")\
               .mean("AEP_MW")\
               .sort(col('DateTime'))
    
    data_length = data.count()
    train_size = int(round(0.7 * data_length,0))
    
    
    ##Add future days to predict
    
    last_day = data.tail(1)[0].__getitem__("Datetime")  # Não sei se é viável
    future_days = pd.date_range(start = last_day, periods = 29)
    sequence_days = list(future_days.strftime("%Y-%m-%d"))[1:-1]
    future = spark.createDataFrame(sequence_days, StringType())
    future.createOrReplaceTempView("future")
    future = spark.sql("SELECT value AS Datetime FROM future")
    future = future.withColumn(data.columns[1],lit(None))
    

    
    df = (data.union(future)).sort(col('Datetime'))
    df = dfZipWithIndex(df,colName="id")
    
    
    # 70% of the real dataset
    #cutoff = '2014-06-08'
    cutoff = train_size
    # Apply forcasting
    global_predictions = (df
                          .groupBy()
                          .apply(fit_pandas_udf)
                          )

/home/lyncoln/spark/python/pyspark/sql/pandas/group_ops.py:81: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [10]:
global_predictions.show()

Index(['ds', 'y'], dtype='object')                                  (0 + 1) / 1]
Index(['ds'], dtype='object')
Initial log joint probability = -31.8657
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       8759.76    0.00335605       1890.93           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       8784.39    0.00162212       143.623      0.8158      0.8158      248   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       8786.88   0.000171334       197.427           1           1      383   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     326       8787.95   9.80295e-05       338.505   4.674e-07       0.001      463  LS failed, Hessian reset 
     380       8788.65   1.91345e-05       93.1063   1.957e-07       0.001      578  LS failed, Hessian rese

+----------+------------------+
|        ds|              yhat|
+----------+------------------+
|2014-06-09| 15679.71994478022|
|2014-06-10|16019.616327382431|
|2014-06-11| 16039.62830482524|
|2014-06-12| 16085.79242202189|
|2014-06-13|15832.072068242376|
|2014-06-14|14656.448318788878|
|2014-06-15|14250.972644239979|
|2014-06-16| 15924.60862734608|
|2014-06-17|16197.115709348192|
|2014-06-18|16152.375409076561|
|2014-06-19|16137.985126126703|
|2014-06-20|15829.371756925782|
|2014-06-21|14605.843465873702|
|2014-06-22| 14160.60679389803|
|2014-06-23|15803.577242323805|
|2014-06-24|16055.237228472808|
|2014-06-25|15999.942582784812|
|2014-06-26|15985.503255891357|
|2014-06-27|15687.290836482642|
|2014-06-28|14484.291398089717|
+----------+------------------+
only showing top 20 rows



In [ ]:
df.filter(col('id')==train_size).show()14484.291398089717